In [107]:
from scikeras.wrappers import KerasClassifier


In [17]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV

In [19]:
data=pd.read_csv("Alphabets_data.csv")
data.head()

,letter,xbox,ybox,width,height,onpix,xbar,ybar,x2bar,y2bar,xybar,x2ybar,xy2bar,xedge,xedgey,yedge,yedgex
0,T,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,I,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,D,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,N,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,G,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10


In [21]:
data.isnull().sum()

letter    0
xbox      0
ybox      0
width     0
height    0
onpix     0
xbar      0
ybar      0
x2bar     0
y2bar     0
xybar     0
x2ybar    0
xy2bar    0
xedge     0
xedgey    0
yedge     0
yedgex    0
dtype: int64

In [23]:
data.duplicated().sum()

1332

In [25]:
data.shape

(20000, 17)

In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   letter  20000 non-null  object
 1   xbox    20000 non-null  int64 
 2   ybox    20000 non-null  int64 
 3   width   20000 non-null  int64 
 4   height  20000 non-null  int64 
 5   onpix   20000 non-null  int64 
 6   xbar    20000 non-null  int64 
 7   ybar    20000 non-null  int64 
 8   x2bar   20000 non-null  int64 
 9   y2bar   20000 non-null  int64 
 10  xybar   20000 non-null  int64 
 11  x2ybar  20000 non-null  int64 
 12  xy2bar  20000 non-null  int64 
 13  xedge   20000 non-null  int64 
 14  xedgey  20000 non-null  int64 
 15  yedge   20000 non-null  int64 
 16  yedgex  20000 non-null  int64 
dtypes: int64(16), object(1)
memory usage: 2.6+ MB


In [29]:
# Normalize numerical features
scaler = MinMaxScaler()
data[['xbox', 'ybox', 'width', 'height', 'onpix', 'xbar', 'ybar', 'x2bar', 'y2bar', 'xybar', 'x2ybar', 'xy2bar', 'xedge', 'xedgey', 'yedge', 'yedgex']] = scaler.fit_transform(data[['xbox', 'ybox', 'width', 'height', 'onpix', 'xbar', 'ybar', 'x2bar', 'y2bar', 'xybar', 'x2ybar', 'xy2bar', 'xedge', 'xedgey', 'yedge', 'yedgex']])

# Encode categorical features
data['letter'] = data['letter'].map({'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25})

In [31]:
data

,letter,xbox,ybox,width,height,onpix,xbar,ybar,x2bar,y2bar,xybar,x2ybar,xy2bar,xedge,xedgey,yedge,yedgex
0,19,0.133333,0.533333,0.200000,0.333333,0.066667,0.533333,0.866667,0.000000,0.400000,0.400000,0.666667,0.533333,0.000000,0.533333,0.000000,0.533333
1,8,0.333333,0.800000,0.200000,0.466667,0.133333,0.666667,0.333333,0.333333,0.266667,0.866667,0.200000,0.600000,0.133333,0.533333,0.266667,0.666667
2,3,0.266667,0.733333,0.400000,0.533333,0.400000,0.666667,0.400000,0.133333,0.400000,0.666667,0.200000,0.466667,0.200000,0.466667,0.200000,0.600000
3,13,0.466667,0.733333,0.400000,0.400000,0.200000,0.333333,0.600000,0.266667,0.400000,0.266667,0.266667,0.666667,0.400000,0.666667,0.133333,0.533333
4,6,0.133333,0.066667,0.200000,0.066667,0.066667,0.533333,0.400000,0.400000,0.400000,0.400000,0.333333,0.600000,0.066667,0.466667,0.333333,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,3,0.133333,0.133333,0.200000,0.200000,0.133333,0.466667,0.466667,0.466667,0.400000,0.400000,0.400000,0.266667,0.133333,0.533333,0.200000,0.466667
19996,2,0.466667,0.666667,0.533333,0.533333,0.266667,0.266667,0.533333,0.400000,0.600000,0.800000,0.600000,0.866667,0.133333,0.600000,0.200000,0.466667
19997,19,0.400000,0.600000,0.400000,0.466667,0.333333,0.400000,0.733333,0.200000,0.466667,0.733333,0.600000,0.333333,0.133333,0.800000,0.133333,0.266667
19998,18,0.133333,0.200000,0.266667,0.133333,0.066667,0.533333,0.466667,0.133333,0.400000,0.666667,0.400000,0.533333,0.066667,0.600000,0.333333,0.533333


In [33]:
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
X = data.drop(columns=["letter"])
y = data["letter"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [35]:
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

In [39]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [47]:

model1 = Sequential()
model1.add(Dense(units=256, activation="relu", input_dim=X_train.shape[1]))
model1.add(Dense(units=len(set(y)), activation="softmax"))
model1.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model1.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)
test_loss, test_accuracy = model1.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy:.4f}")


C:\Users\bojja\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.2081 - loss: 2.9729 - val_accuracy: 0.5397 - val_loss: 2.0410
Epoch 2/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5386 - loss: 1.8658 - val_accuracy: 0.6187 - val_loss: 1.5572
Epoch 3/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6317 - loss: 1.4679 - val_accuracy: 0.6531 - val_loss: 1.3459
Epoch 4/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6724 - loss: 1.2824 - val_accuracy: 0.6841 - val_loss: 1.2291
Epoch 5/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6949 - loss: 1.1811 - val_accuracy: 0.7041 - val_loss: 1.1593
Epoch 6/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7128 - loss: 1.1038 - val_accuracy: 0.7188 - val_loss: 1.1168
Epoch 7/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7217 - loss: 1.0539 - val_accuracy: 0.7369 - val_loss: 1.0509
Epoch 8/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7408 - loss: 0.9906 - val_accuracy: 0.

In [49]:
y_pred = model1.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
accuracy = accuracy_score(y_test, y_pred_classes)
precision = precision_score(y_test, y_pred_classes, average='weighted')
recall = recall_score(y_test, y_pred_classes, average='weighted')
f1 = f1_score(y_test, y_pred_classes, average='weighted')
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Accuracy: 0.76575
Precision: 0.7753082137267502
Recall: 0.76575
F1-score: 0.766432766577731


In [111]:
from keras import optimizers

from sklearn.model_selection import GridSearchCV
#from keras.wrappers.scikit_learn import KerasRegressor

In [109]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

# Create a KerasClassifier
model = KerasClassifier(build_fn=create_model, verbose=0)

# Define hyperparameters to tune
param_grid = {
    'epochs': [50, 100],
    'batch_size': [32, 64],
    'neurons1': [64, 128],
    'neurons2': [32, 64]
}

# Create a GridSearchCV object
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)

# Fit the GridSearchCV object to the data
grid_search.fit(X_train, y_train)

# Print the best parameters and score
print("Best parameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)

ValueError: Invalid parameter neurons1 for estimator KerasClassifier.
This issue can likely be resolved by setting this parameter in the KerasClassifier constructor:
`KerasClassifier(neurons1=64)`
Check the list of available parameters with `estimator.get_params().keys()`

In [ ]:
# hyperparameter Tuning

In [51]:
learning_rates = [0.01, 0.001, 0.0001]
num_hidden_layers = [1, 2, 3]
neurons_per_layer = [32, 64, 128]
activation_functions = ['relu', 'tanh', 'sigmoid']

for lr in learning_rates:
    for num_layers in num_hidden_layers:
        for neurons in neurons_per_layer:
            for activation in activation_functions:
                # Create a new model instance for each combination
                model = Sequential()

                # Build the hidden layers with correct parameter passing
                for _ in range(num_layers):  # Create num_layers hidden layers
                    model.add(Dense(neurons, activation=activation))

                # Output layer for 26 alphabet classes with softmax activation
                model.add(Dense(len(set(y_train)), activation='softmax'))  # Use y_train

                # Compile the model
                model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

                # Train the model with validation split
                model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

                # Evaluate the model on test set
                test_loss, test_accuracy = model.evaluate(X_test, y_test)

                # Print results with clear formatting
                print(f"Test Accuracy: {test_accuracy:.4f} (Learning Rate: {lr}, Hidden Layers: {num_layers}, Neurons: {neurons}, Activation: {activation})")

Epoch 1/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.1048 - loss: 3.1862 - val_accuracy: 0.3212 - val_loss: 2.8095
Epoch 2/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3390 - loss: 2.6651 - val_accuracy: 0.4603 - val_loss: 2.2983
Epoch 3/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4840 - loss: 2.1915 - val_accuracy: 0.5359 - val_loss: 1.9788
Epoch 4/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5401 - loss: 1.9098 - val_accuracy: 0.5706 - val_loss: 1.7769
Epoch 5/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5864 - loss: 1.7155 - val_accuracy: 0.6006 - val_loss: 1.6352
Epoch 6/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6070 - loss: 1.5861 - val_accuracy: 0.6037 - val_loss: 1.5390
Epoch 7/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6249 - loss: 1.4811 - val_accuracy: 0.6253 - val_loss: 1.4589
Epoch 8/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6373 - loss: 1.4110 - val_accuracy: 0.

# Final ANN Model

In [71]:
learning_rate = 0.0001
num_hidden_layers = 3
neurons_per_layer = 128
activation_function = 'tanh'

# Create the final model
model_final = Sequential()

# Build hidden layers
for _ in range(num_hidden_layers):
    model_final.add(Dense(neurons_per_layer, activation=activation_function))

# Output layer for 26 alphabet classes
model_final.add(Dense(len(set(y_train)), activation='softmax'))

# Compile the model
model_final.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model (you can adjust epochs, batch_size, etc. as needed)
model_final.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)
test_loss, test_accuracy = model_final.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy:.4f}")

Epoch 1/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.3843 - loss: 2.2721 - val_accuracy: 0.6947 - val_loss: 1.1227
Epoch 2/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7103 - loss: 1.0335 - val_accuracy: 0.7450 - val_loss: 0.9277
Epoch 3/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7666 - loss: 0.8369 - val_accuracy: 0.7781 - val_loss: 0.7660
Epoch 4/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7925 - loss: 0.7008 - val_accuracy: 0.8181 - val_loss: 0.6575
Epoch 5/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8341 - loss: 0.5834 - val_accuracy: 0.8394 - val_loss: 0.5731
Epoch 6/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8494 - loss: 0.5218 - val_accuracy: 0.8456 - val_loss: 0.5154
Epoch 7/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8703 - loss: 0.4519 - val_accuracy: 0.8522 - val_loss: 0.4902
Epoch 8/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8777 - loss: 0.4236 - val_accuracy: 0.

In [103]:
y_pred = model_final.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
accuracy = accuracy_score(y_test, y_pred_classes)
precision = precision_score(y_test, y_pred_classes, average='weighted')
recall = recall_score(y_test, y_pred_classes, average='weighted')
f1 = f1_score(y_test, y_pred_classes, average='weighted')
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Accuracy: 0.9055
Precision: 0.9079866155189958
Recall: 0.9055
F1-score: 0.9056574467926257


In [ ]:
y_train

In [91]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV

In [93]:
model_final_ = KerasClassifier(build_fn=model_final, verbose=0)

In [95]:
from random import randint
import random
from scipy.stats import randint

In [99]:
random_search = RandomizedSearchCV(estimator=model_final_,param_distributions=param_dist, scoring='accuracy')

In [101]:
random_search.fit(X_train, y_train)

ValueError: Invalid parameter activation for estimator KerasClassifier.
This issue can likely be resolved by setting this parameter in the KerasClassifier constructor:
`KerasClassifier(activation=sigmoid)`
Check the list of available parameters with `estimator.get_params().keys()`